<h1 align=center>Stock Analysis</h1>

In [1]:
# from phi.agent import Agent
# from phi.model.openai import OpenAIChat
# from phi.tools.yfinance import YFinanceTools

# finance_agent = Agent(
#     name="Finance Agent",
#     model=OpenAIChat(id="gpt-4o"),
#     tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
#     instructions=["Use tables to display data"],
#     show_tool_calls=True,
#     markdown=True,
# )
# finance_agent.print_response("Summarize analyst recommendations for NVDA", stream=True)

### Building an Agentic Financial Analyst with LangGraph and OpenAI

What this Agentic Financial Analyst will do?
- Fetches stock price data using Yahoo Finance.
- Calculates technical indicators like RSI, MACD, VWAP, and more.
- Evaluate financial metrics such as P/E ratio, Debt-to-Equity, and Profit Margins.
- Provides a structured, AI-generated analysis using OpenAI’s powerful language models.
`Tools We’ll Use`
- LangGraph: A library for orchestrating tools and building conversational agents.
- OpenAI GPT-4: For generating intelligent and structured financial insights.
- yfinance: To retrieve stock prices and financial ratios.
- ta (Technical Analysis Library): For calculating key technical indicators.
- Python libraries: pandas, dotenv, and datetime for data manipulation and environment setup.

#### Step 1: Setting Up the Environment

#### Step 2: Tools for Analyst
Fetching Stock Prices: This tool fetches the stock’s historical data and computes several technical indicators.

In [2]:
from typing import Union, Dict, Set, List, TypedDict, Annotated
import pandas as pd
from langchain_core.tools import tool
import yfinance as yf
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.volume import volume_weighted_average_price

@tool
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicator for a given ticker."""
    try:
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=24*3),
            end=dt.datetime.now(),
            interval='1wk'
        )
        df= data.copy()
        data.reset_index(inplace=True)
        data.Date = data.Date.astype(str)
        
        indicators = {}
        
        rsi_series = RSIIndicator(df['Close'], window=14).rsi().iloc[-12:]
        indicators["RSI"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in rsi_series.dropna().to_dict().items()}
        
        sto_series = StochasticOscillator(
            df['High'], df['Low'], df['Close'], window=14).stoch().iloc[-12:]
        indicators["Stochastic_Oscillator"] = {
                    date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in sto_series.dropna().to_dict().items()}

        macd = MACD(df['Close'])
        macd_series = macd.macd().iloc[-12:]
        indicators["MACD"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in macd_series.to_dict().items()}
        
        macd_signal_series = macd.macd_signal().iloc[-12:]
        indicators["MACD_Signal"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in macd_signal_series.to_dict().items()}
        
        vwap_series = volume_weighted_average_price(
            high=df['High'], low=df['Low'], close=df['Close'], 
            volume=df['Volume'],
        ).iloc[-12:]
        indicators["vwap"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in vwap_series.to_dict().items()}
        
        return {'stock_price': data.to_dict(orient='records'),
                'indicators': indicators}

    except Exception as e:
        return f"Error fetching price data: {str(e)}"

Financial Ratios: This tool retrieves key financial health ratios.

In [3]:
@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"

#### Step 3: Building the LangGraph
LangGraph allows us to orchestrate tools and manage conversational logic efficiently.

1. Defining the Graph
We start by defining a StateGraph to manage the flow:

In [4]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]
    stock: str
    
graph_builder = StateGraph(State)

2. Defining OpenAI and Binding Tools

We integrate the tools into LangGraph and create a feedback loop for analysis

In [5]:
# import dotenv
# dotenv.load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

tools = [get_stock_prices, get_financial_metrics]
llm_with_tool = llm.bind_tools(tools)

3. Analyst Node

The prompt ensures the AI understands its role and delivers structured output.

In [6]:
from langchain.schema import SystemMessage


FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, and financial metrics. Your task is to provide a comprehensive summary of the fundamental analysis for a given stock.

You have access to the following tools:
1. **get_stock_prices**: Retrieves the latest stock price, historical price data and technical Indicators like RSI, MACD, Drawdown and VWAP.
2. **get_financial_metrics**: Retrieves key financial metrics, such as revenue, earnings per share (EPS), price-to-earnings ratio (P/E), and debt-to-equity ratio.

### Your Task:
1. **Input Stock Symbol**: Use the provided stock symbol to query the tools and gather the relevant information.
2. **Analyze Data**: Evaluate the results from the tools and identify potential resistance, key trends, strengths, or concerns.
3. **Provide Summary**: Write a concise, well-structured summary that highlights:
    - Recent stock price movements, trends and potential resistance.
    - Key insights from technical indicators (e.g., whether the stock is overbought or oversold).
    - Financial health and performance based on financial metrics.

### Constraints:
- Use only the data provided by the tools.
- Avoid speculative language; focus on observable data and trends.
- If any tool fails to provide data, clearly state that in your summary.

### Output Format:
Respond in the following format:
"stock": "<Stock Symbol>",
"price_analysis": "<Detailed analysis of stock price trends>",
"technical_analysis": "<Detailed time series Analysis from ALL technical indicators>",
"financial_analysis": "<Detailed analysis from financial metrics>",
"final Summary": "<Full Conclusion based on the above analyses>"
"Asked Question Answer": "<Answer based on the details and analysis above>"

Ensure that your response is objective, concise, and actionable."""

def fundamental_analyst(state: State):
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company=state['stock'])),
    ]  + state['messages']
    return {
        'messages': llm_with_tool.invoke(messages)
    }

graph_builder.add_node('fundamental_analyst', fundamental_analyst)
graph_builder.add_edge(START, 'fundamental_analyst')

4. Adding tool to graph and compile

In [7]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition



graph_builder.add_node(ToolNode(tools))
graph_builder.add_conditional_edges('fundamental_analyst', tools_condition)
graph_builder.add_edge('tools', 'fundamental_analyst')

graph = graph_builder.compile()

5. Executing the Graph

In [9]:
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': 'NVIDIA Corp'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (call_eNANVE1DE7nZPMuWMxNA37VN)
 Call ID: call_eNANVE1DE7nZPMuWMxNA37VN
  Args:
    ticker: NVDA
  get_financial_metrics (call_hfPDp9EuPhB7KsQJBd6F4sYa)
 Call ID: call_hfPDp9EuPhB7KsQJBd6F4sYa
  Args:
    ticker: NVDA
================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 26.924759, "price_to_book": 34.227093, "debt_to_equity": 12.946, "profit_margins": 0.55848}
================================== Ai Message ==================================

It seems there was an issue retrieving the stock price and technical indicators for NVIDIA Corp (NVDA). However, I was able to gather some key financial metrics. Here's the summary based on the available data:

"stock": "NVDA",
"price_analysis": "Unable to retrieve 

### 

2. ### 